# Predicting from Sound
> A Neural Networks project by Aleksander Nikolajev, Kayahan Kaya and Severin Brunner

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- sticky_rank: 1

## Introduction
Gaining information from sounds is a fundamental human ability: We can detect and identify objects just from our hearing as well as estimate the direction and distance of that object. 
Writing software with the same abilities is a difficult task due to the enormous complexity of audio signals. Applying machine learning, in particular neural networks, is the most promising approach to meet this challenge. 
In this project, we are researching common methods to deploy neural networks for prediction from sound and are creating our own neural network that is able to extract certain information from audio samples. In particular, we are trying to predict the source of a sound as well as the distance between source and microphone.


## Audio theory
In this section, essential theoretical elements of audio analysis are introduced, which we later utilize for our network.

### Pulse-code modulation (PCM)
In order to store an analog audio signal in memory, it has to be digitized by applying sampling and quantization. Sampling refers to measuring the signal values at specific timesteps, which transforms the original time-continuous signal into a time-discrete one. Quantization implies mapping the continuous signal values to discrete values in a specific range, e.g. 16 bits.

![](https://upload.wikimedia.org/wikipedia/commons/b/bf/Pcm.svg "Figure 1: Sampling and quantization of an analog signal (red) with 4-bit PCM, resulting in a time-discrete and value-discrete signal (blue).")

PCM is a format for storing uncompressed audio signals. It simply contains an array of values that have been produced by sampling and quantizing an analog signal. It has two basic properties:  The sampling rate (how many samples per second were taken) and the bit depth (the number of bits per sample value), which determines the resolution. A typical sampling rate is 44.1 kHz (e.g. CDs), and 16 bits is a common choice for the bit depth.



### Spectograms

A spectrogram is a visualiziaton of the frequency spectrum of a signal over time. The frequency spectrum represents the signal strength of the various frequencies present in the signal. It can be calculated by applying a fourier transform to the signal.
The spectogram is depicted as a heat map, which means the intensity at a specific frequency and time is expressed through the color.
![](clarinette_spectogram.png "Figure 2: Spectrogram of a recording of a clarinet playing a note. The bottom line is at the frequency of the keynote, the higher lines are the harmonics. The clarinet starts playing at 0.4 seconds.")

### MFCC (Mel-Frequency Cepstral Coefficients)
For audio analysis, it often makes sense to extract certain features from the raw audio signal, like the signal energy or the spectogram. As a feature, the MFCCs represent the entire frequency spectrum compactly with few values (e.g. 40), which approximates the human auditory system more closely. This has proven useful for applications like speech or song recognition.

## The Dataset

For our project, we used an [FSD50k Zenodo](https://zenodo.org/record/4060432) audio dataset, commonly used in kaggle competitions. It has over 50k audio samples with 200 classes, with one audio sample having multiple classes. The FSD50k dataset was utilized for the classification part of this project.


### Creating our own dataset

While the FSD50k dataset has plenty of sound samples, it cannot be used for distance predictions on its own. Using the FSD50k dataset, we recorded approximately 3000 audio samples from different distances in one room. Note that the room wasn't soundproof, therefore some background noise was inevitably included.

Figure 3 demonstrates the recording process. We placed the microphone in a distance d of either 1 meter or 2 meters away from the speaker. To properly record the audio samples, we used a distributed client-server architecture that would notify the microphone client when the source audio would start playing the sound.

![](dataset_recording_draft.png "Figure 3: Draft of the recording process. A PC connected to a speaker plays the samples, while a laptop records it with a microphone from a certain distance d, in our case 1 and 2 meters. The PC signals the laptop when it starts and stops playing each sample over a socket connection, so the laptop can start and stop recording its samples accordingly.") 

It could be argued that it would have been easier to play and record the sound from the same computer. This however was infeasible for us, as we didn't have the proper equipment necessary to accomplish this.

The final distance dataset contained approximately 3000 audio samples from over 100 different classes.

### Data Preprocessing

#### Polishing the dataset
During the creation of the dataset, we noticed that the distributed client-server architecture would sometimes start recording too late, as the audio file was so short that it could not send the message in time. Because recording again was deemed wasteful of our precious time, we decided to filter out the non-existant audio samples. The filtered dataset was still over 2500 audio samples large.

As mentioned before, a single audio sample could have multiple classes. While this is not an issue with distance prediction, this provided an extra challenge in the classification part of the project. A first step to manage this was to accomplish that all audio samples have the same amount of classes.  Figure 4 shows an example of the process.

![](figure_prep.png "Figure 4")

#### Normalization
One way to normalize audio signals as part of data preparation is to set the RMS (root mean square) of all audio signals to a fixed value {% fn 2 %}. This can be done by calculating the RMS for each audio sample and then dividing all the sample values by the RMS. Normalizing the RMS can be interpreted as ensuring that each audio sample has the same average power output.

Another way is to use the common min/max normalization, which we ended up using in the project due to better results.

#### Removing background noise
In order to provide as much meaningful information as possible to the network, any unintended parts of the audio signal need to be filtered out. The recording process injected a certain amount of noise into the samples, which confound the network and therefore should be ideally eliminated. For this task, we tested several common background noise removal approaches for their effectiveness.

The method of short-term energy for noise cancellation is commonly used for voice detection {% fn 3 %}. It detects the noisy parts of a signal because they have less energy than the voice parts. The noisy parts can then be removed from the signal. 

For noise cancellation in vibration signals, autocorrelation has been shown to be a useful tool {% fn 4 %}. The vibration is correlated to itself, however the noise is neither correlated to itself nor the signal. Therefore repeatedly applying the autocorrelation operation to the signal reduces the noise more and more while the vibration is not decreased. 

These two concepts however rely on a priori estimates of the signal, i.e. the signal being a voice or a vibration. Another method, Adaptive Noise Cancellation (ANC), deals with noise without any of these assumptions. ANC is achieved by introducing a canceling antinoise wave through secondary sources. These secondary sources are interconnected through an electronic system using a specific signal processing algorithm for the particular cancellation scheme. To implement ANC,we wrote an algorithm in matlab for a single channel feed-forward active noise control system. We then sent a sequence of training data to estimate the noise before the setup of noise cancellation, and trained the filter with white gaussian noise.

![](ANC_error.jpg "Figure 5: Calculated error of a feedforward ANC")
We took one of the longer noisy signals from our  dataset and tried to eliminate the noise with the ANC feedforward method. In figure 5, our target is the error. Here error is original signal subtracted by filtered signal. Our target is to minimize error as much as we can. From figure 1 filter is almost making error zero in  80 iteration then exit the loop ??

Figure 6 shows the difference between original signal and noise eliminated signal.

![](ANC_difference.png "Figure 6: Original vs. denoised signal")

We applied the different denoising techniques, which indeed resulted in less interference for some samples. However due to the big diversity in our dataset the findings were inconsistent. What worked for one kind of signal effected another kind of signal negatively. Because considering each signal individually would be far too costly, we ultimately decided not to deploy any denoising techniques for now and instead let our network deal with the background noise. In future work, dealing with this could lead to accuracy improvements.

## Creating the network


### Model implementation
Due to having two different problem statements, we opted on using two different models.  In this section, we will discuss the two models that were implemented, as well as their performance.


#### Python framework

We are using Keras as the deep learning library to construct our network.

For audio processing, Librosa is a suitable library. It provides several functions to extract features from audio data, e.g. for creating spectograms, calculating the MFCCs and performing a fourier transform.

Other libraries, such as SKlearn and pandas, help with the data processing and KFold model fitting.

#### Network input
The audio prediction portion of this project could easily boil down to image classification by using some form of image representation of the audio samples, for example images of the raw audio signal or the corresponding spectrogram. Figure 6 shows some of these raw audio inputs.

![](figure_example.png "Figure 6")

Instead, however, we opted to extract MFCC features using Librosa. Librosa returns the MFCC features from a signal over time in an arrays, which we can then use for classification. As MFCC is a more compact representation of an audio signal, training is a lot faster due to the network having to process a smaller amount of data.


For the distance portion, we decided to try a different approach. We would create a spectogram of each audio sample and make this an image classification problem. There are primarly two reasons for adopting this approach:

1. The previous approach yielded petty results in terms of accuracy.
2. We wanted to compare feature extraction and image classification approaches.

#### Classification
For the classification, we used a classic Convolution Neural Network (CNN).
This decision was inspired by a popular blog from a kaggle competition called ["Beginners guide to audio data"](https://www.kaggle.com/fizzbuzz/beginner-s-guide-to-audio-data/). 

The convolutional network model can be seen in Figure 7.

![](class1.png "Figure 7")

This model, however, proved to perform very poorly, barely hitting 1% validation accuracy, even though the training accuracy proved to be at 28%. We believe the cause for this is due to the small size of the dataset (2500) and the abundance of classes, leaving little room for error.

##### Tackling multi-label classification
In addition to the problems mentioned above, this model isn't suited to predict multiple labels. Jason Brownlee and his guide "Multi-Label Classification of Satellite Phootos of the Amazon Rainforest" gave us an idea: what we can do is map all n labels to integers and store an n-element vector for each audio file. This vector contains 0 for labels that don't apply to the audio file, and 1 for labels that do. This corresponds to one hot encoding. After utilizing this technique, the predicting process is quite straightforward, resembling an Image Classification task.

##### Evaluation
As we are dealing with a multi-class classification task, commonly used performance metrics for binary classification tasks aren't suitable. For example, the classic F1 score the mean of precision and recall. The precision describes how good the model is at predicting the positive outcome, and the recall quantifies the model's ability to predict positive samples as positive. This however doesn't make sense for predicting multiple classes, as there is no clear positive and negative class. 

The so-called F-Beta metric overcomes this by first calculating prediction and recall for each class in a one vs rest manner and then averaging them over all the classes. A constant Beta is utilized in order to weigh precision and recall differently. We are using the common choice of two for Beta, which makes recall valued twice as highly as precision. 
The F-Beta metric is calculated as follows:

*F-Beta = (1 + Beta^2) x (precision x recall) / (Beta^2 x precision + recall)*


![](second_model.png "Figure x")

The results in the above figure show two graphs, cross entropy loss and F-Beta. This figure explains that it is quite heavily overfitting the data, however, the F-Beta value is close to the ideal, signifying the

#### Distance
For the distance prediction model, we fit it with images of all the audio spectograms. As for the model itself, we used a new concept for us - CRNN. 

The concept to use CRNN for distance classification was proposed by Mariam Yiwere et al. in ["Sound Source Distance Estimation Using Deep Learning: An Image Classification Approach](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6982911/). A CRNN enables a model to learn bot hthe spectral and temporal features and relationships effectively. 

The CRNN model can be see in Figure 9.
As a loss function, we used cross entropy, and the Adam optimizer is used for training.

![](CRNN.png "Figure 9")

The hope for this project was that it would predict better than a coin flip, because we only had the time to record two different distances (1 and 2). Our hope was fulfilled, and the validation accuracy for this model was 53%.





## Conclusion and future work

We have created 2 models for two different problem statements - that being audio classification and source distance prediction. For these tasks, we researched multiple to solve these problems, and explain concepts such as CRNN and audio feature extraction. While the results are not great, we believe that the methods we used are intuitive and correct.

For future work, we would mention the idea of using different kinds of noise cancellation techniques that were researched throughout this project. One of those ideas is called Short-Term autocorellation Method, which capsuls the pitch point of signal. 

### Contributions of the team members
Aleksander was responsible for creating the classification network as well as recording the dataset. 

Kayahan managed the project, thereby taking a directive role. He also did extensive research about techniques to eliminate background noise. 

Severin created and maintained the blog and undertook debugging tasks.

### Source code
[Github link](https://github.com/TheGadgeteer/Predicting-from-Sound-Blog/tree/master/Code)

## References

{{ 'https://www.kaggle.com/fizzbuzz/beginner-s-guide-to-audio-data/' | fndetail: 1}} USED?

{{ 'https://arxiv.org/pdf/2003.04210.pdf' | fndetail: 2}}

{{ 'https://www.researchgate.net/publication/263354982_A_Hierarchical_Framework_Approach_for_Voice_Activity_Detection_and_Speech_Enhancement' | fndetail: 3 }}

{{ 'https://www.sciencedirect.com/science/article/pii/S2590123020300426' | fndetail: 4 }}